# Gel 

> An implementation of LangChain vectorstore abstraction using `gel` as the backend.

The code lives in an integration package called: [langchain_gel](https://github.com/geldata/langchain-gel).

## Setup

First install the package:

In [ ]:
! pip install -qU gel langchain-gel 

In [ ]:
! gel project init --non-interactive

```bash
gel project init --server-instance <org-name>/<instance-name>
```

In [14]:
schema_content = """using extension pgvector;
                                    
module default {
    scalar type EmbeddingVector extending ext::pgvector::vector<1536>;

    type Record {
        required collection: str;
        text: str;
        embedding: EmbeddingVector; 
        external_id: str {
            constraint exclusive;
        };
        metadata: json;

        index ext::pgvector::hnsw_cosine(m := 16, ef_construction := 128)
            on (.embedding)
    } 
}"""

with open("dbschema/default.gel", "w") as f:
    f.write(schema_content)

In [ ]:
! gel migration create --non-interactive
! gel migrate

## Instantiation

import EmbeddingTabs from "@theme/EmbeddingTabs";

<EmbeddingTabs/>


In [2]:
# | output: false
# | echo: false
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [3]:
from langchain_gel import GelVectorStore


vector_store = GelVectorStore(
    embeddings=embeddings,
)

## Manage vector store

### Add items to vector store

Note that adding documents by ID will over-write any existing documents that match that ID.

In [4]:
from langchain_core.documents import Document

docs = [
    Document(
        page_content="there are cats in the pond",
        metadata={"id": "1", "location": "pond", "topic": "animals"},
    ),
    Document(
        page_content="ducks are also found in the pond",
        metadata={"id": "2", "location": "pond", "topic": "animals"},
    ),
    Document(
        page_content="fresh apples are available at the market",
        metadata={"id": "3", "location": "market", "topic": "food"},
    ),
    Document(
        page_content="the market also sells fresh oranges",
        metadata={"id": "4", "location": "market", "topic": "food"},
    ),
    Document(
        page_content="the new art exhibit is fascinating",
        metadata={"id": "5", "location": "museum", "topic": "art"},
    ),
    Document(
        page_content="a sculpture exhibit is also at the museum",
        metadata={"id": "6", "location": "museum", "topic": "art"},
    ),
    Document(
        page_content="a new coffee shop opened on Main Street",
        metadata={"id": "7", "location": "Main Street", "topic": "food"},
    ),
    Document(
        page_content="the book club meets at the library",
        metadata={"id": "8", "location": "library", "topic": "reading"},
    ),
    Document(
        page_content="the library hosts a weekly story time for kids",
        metadata={"id": "9", "location": "library", "topic": "reading"},
    ),
    Document(
        page_content="a cooking class for beginners is offered at the community center",
        metadata={"id": "10", "location": "community center", "topic": "classes"},
    ),
]

vector_store.add_documents(docs, ids=[doc.metadata["id"] for doc in docs])

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

### Delete items from vector store

In [5]:
vector_store.delete(ids=["3"])

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent. 

### Filtering Support

The vectorstore supports a set of filters that can be applied against the metadata fields of the documents.

| Operator | Meaning/Category        |
|----------|-------------------------|
| \$eq      | Equality (==)           |
| \$ne      | Inequality (!=)         |
| \$lt      | Less than (&lt;)           |
| \$lte     | Less than or equal (&lt;=) |
| \$gt      | Greater than (>)        |
| \$gte     | Greater than or equal (>=) |
| \$in      | Special Cased (in)      |
| \$nin     | Special Cased (not in)  |
| \$between | Special Cased (between) |
| \$like    | Text (like)             |
| \$ilike   | Text (case-insensitive like) |
| \$and     | Logical (and)           |
| \$or      | Logical (or)            |

### Query directly

Performing a simple similarity search can be done as follows:

In [6]:
results = vector_store.similarity_search(
    "kitty", k=10, filter={"id": {"$in": ["1", "5", "2", "9"]}}
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* there are cats in the pond [{'id': '1', 'topic': 'animals', 'location': 'pond'}]
* ducks are also found in the pond [{'id': '2', 'topic': 'animals', 'location': 'pond'}]
* the new art exhibit is fascinating [{'id': '5', 'topic': 'art', 'location': 'museum'}]
* the library hosts a weekly story time for kids [{'id': '9', 'topic': 'reading', 'location': 'library'}]


If you provide a dict with multiple fields, but no operators, the top level will be interpreted as a logical **AND** filter

In [7]:
vector_store.similarity_search(
    "ducks",
    k=10,
    filter={"id": {"$in": ["1", "5", "2", "9"]}, "location": {"$in": ["pond", "market"]}},
)

[Document(id='2', metadata={'id': '2', 'topic': 'animals', 'location': 'pond'}, page_content='ducks are also found in the pond'),
 Document(id='1', metadata={'id': '1', 'topic': 'animals', 'location': 'pond'}, page_content='there are cats in the pond')]

In [8]:
vector_store.similarity_search(
    "ducks",
    k=10,
    filter={
        "$and": [
            {"id": {"$in": ["1", "5", "2", "9"]}},
            {"location": {"$in": ["pond", "market"]}},
        ]
    },
)

[Document(id='2', metadata={'id': '2', 'topic': 'animals', 'location': 'pond'}, page_content='ducks are also found in the pond'),
 Document(id='1', metadata={'id': '1', 'topic': 'animals', 'location': 'pond'}, page_content='there are cats in the pond')]

If you want to execute a similarity search and receive the corresponding scores you can run:

In [9]:
results = vector_store.similarity_search_with_score(query="cats", k=1)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.471662] there are cats in the pond [{'id': '1', 'topic': 'animals', 'location': 'pond'}]


For a full list of the different searches you can execute on a `PGVector` vector store, please refer to the [API reference](https://python.langchain.com/api_reference/postgres/vectorstores/langchain_postgres.vectorstores.PGVector.html).

### Query by turning into retriever

You can also transform the vector store into a retriever for easier usage in your chains. 

In [10]:
retriever = vector_store.as_retriever(search_kwargs={"k": 1})
retriever.invoke("kitty")

[Document(id='1', metadata={'id': '1', 'topic': 'animals', 'location': 'pond'}, page_content='there are cats in the pond')]

## Usage for retrieval-augmented generation

For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:

- [Tutorials](/docs/tutorials/)
- [How-to: Question and answer with RAG](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [Retrieval conceptual docs](https://python.langchain.com/docs/concepts/retrieval)